# Oblig2:

- Student Number: S374918
- Student Name: Alex McCorkle

(Lecturer said it was OK to do it alone)

### Use Case: Tesla Stock

### Machine Learning Algorithm: Random Forest Regression

For this assignment I've decided to try out implementing a Random Forest Regression model to be able to accurately predict the stock price on a specific date. After reading around and watching a few videos on the differences between a few algorithms, I found that Random Forest seems to be a popular ML algorithm for predicting complex data, such as the Stock market. It is also a model that is can be quite accurate and there are findings to back this up. Khan et al. (2023) found when comparing 9 different machine learning models that Random Forest outperformed the other models when forecasting the stock market.



Khan, A. H., Shah, A., Ali, A., Shahid, R., Zahid, Z. U., Sharif, M. U., Jan, T., & Zafar, M. H. (2023). A performance comparison of machine learning models for stock market prediction with novel investment strategy. PloS one, 18(9), e0286362. https://doi.org/10.1371/journal.pone.0286362

In [9]:
# Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load data:
df = pd.read_csv("TSLA.csv")

df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2845.000000,2845.000000,2845.000000,2845.000000,2845.000000,2.845000e+03
mean,105.797199,107.959321,103.485805,105.853488,105.853488,3.145634e+07
std,188.691743,192.434544,184.592519,188.788993,188.788993,2.845125e+07
min,3.228000,3.326000,2.996000,3.160000,3.160000,5.925000e+05
25%,10.352000,10.748000,10.132000,10.400000,10.400000,1.252200e+07
50%,45.868000,46.493999,45.099998,45.902000,45.902000,2.482100e+07
75%,65.000000,66.251999,63.993999,65.234001,65.234001,4.014650e+07
max,891.380005,900.400024,871.599976,883.090027,883.090027,3.046940e+08


In [8]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2840,2021-10-08,796.210022,796.380005,780.909973,785.489990,785.489990,16711100
2841,2021-10-11,787.650024,801.239990,785.500000,791.940002,791.940002,14200300
2842,2021-10-12,800.929993,812.320007,796.570007,805.719971,805.719971,22020000
2843,2021-10-13,810.469971,815.409973,805.780029,811.080017,811.080017,14120100
2844,2021-10-14,815.489990,820.250000,813.349976,818.320007,818.320007,12203200
